In [10]:
import imageio as io
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping
import time
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
im = io.imread(r"d:\Documents\Nexmedis Salsa\chest_xray\test\PNEUMONIA_BACTERIA\person78_bacteria_381.jpeg")
im.shape

C:\Users\Muhammad Asyarie\AppData\Local\Temp\ipykernel_28136\2614593021.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = io.imread(r"d:\Documents\Nexmedis Salsa\chest_xray\test\PNEUMONIA_BACTERIA\person78_bacteria_381.jpeg")


(536, 824)

Preprocessing - Median Filter + Contrast Enhancement

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image, ImageEnhance, ImageFilter
import matplotlib.pyplot as plt

# Path gambar yang ingin diproses
input_folder = r"d:\Documents\Nexmedis Salsa\chest_xray"
output_folder = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing"

# Buat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

# 2. Preprocessing: Resize, Normalisasi, dan Konversi ke Grayscale
def preprocess_image(image):
    # Resize gambar ke ukuran seragam, misal 224x224
    resized_image = cv2.resize(image, (224, 224))
    
    # Konversi ke grayscale jika gambar berwarna
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # Normalisasi gambar ke rentang 0-1
    normalized_image = grayscale_image / 255.0
    
    return normalized_image

# Fungsi untuk memproses setiap gambar dalam folder dan subfolder
def process_and_save_images(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        # Dapatkan path relatif dari subfolder ke input_folder
        relative_path = os.path.relpath(root, input_folder)
        
        # Buat folder yang sesuai di output_folder
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)
        
        for filename in files:
            if filename.endswith('.jpeg') or filename.endswith('.jpg') or filename.endswith('.png'):
                # Path lengkap ke file gambar
                img_path = os.path.join(root, filename)
                
                # Memuat gambar menggunakan OpenCV
                image = cv2.imread(img_path)
                
                # Terapkan preprocessing
                preprocessed_image = preprocess_image(image)
                global median_filtered_image, sharpened_image
                # Menerapkan Median Filter menggunakan OpenCV
                median_filtered_image = cv2.medianBlur(np.float32(preprocessed_image), 5)
                
                # Convert OpenCV image (numpy array) to PIL Image
                pil_image = Image.fromarray(np.uint8(median_filtered_image * 255))
                
                # Enhance contrast
                enhancer = ImageEnhance.Contrast(pil_image)
                image_contrast = enhancer.enhance(2)
                
                # Sharpen the image
                sharpened_image = image_contrast.filter(ImageFilter.SHARPEN)
                
                # Save the processed image to the corresponding output subfolder
                output_path = os.path.join(output_subfolder, filename)
                sharpened_image.save(output_path)
                print(f"Processed and saved: {output_path}")
# Panggil fungsi untuk memproses gambar
process_and_save_images(input_folder, output_folder)

PREPROCESSING HIGHPASS FILTER

In [ ]:
import cv2
import numpy as np
import os

# Path gambar yang ingin diproses
input_folder = r"C:\Users\Salsabiela\OneDrive - Institut Teknologi Sepuluh Nopember\Nexmedis\chest_xray\val\PNEUMONIA"
output_folder = r"D:\SALSA NITIP\Nexmedis\preprocessing\PNEUMONIA\bacteria"
# Buat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

# Preprocessing
def preprocess_image(image):
    # Konversi gambar menjadi grayscale jika belum
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Terapkan Sobel filter
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=1)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=1)
    sobel = np.sqrt(sobel_x**2 + sobel_y**2)

    # Konversi hasil Sobel ke format uint8
    sobel = cv2.convertScaleAbs(sobel)

    # Threshold untuk mengisolasi tepi yang kuat
    _, thresholded = cv2.threshold(sobel, 100, 255, cv2.THRESH_BINARY)

    # Find contours dari simbol "R"
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if 10 < w < 100 and 10 < h < 100:  # Ukuran yang sesuai dengan "R"
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), -1)  # Menghapus "R"
    
    return image  # Mengembalikan gambar yang telah diproses

# Fungsi untuk memproses setiap gambar dalam folder dan subfolder
def process_and_save_images(input_folder, output_folder):
    for root, dirs, files in os.walk(input_folder):
        # Dapatkan path relatif dari subfolder ke input_folder
        relative_path = os.path.relpath(root, input_folder)
        
        # Buat folder yang sesuai di output_folder
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)
        
        for filename in files:
            if filename.endswith('.jpeg') or filename.endswith('.jpg') or filename.endswith('.png'):
                # Path lengkap ke file gambar
                img_path = os.path.join(root, filename)
                
                # Memuat gambar menggunakan OpenCV
                image = cv2.imread(img_path)
                
                # Pastikan gambar berhasil dimuat sebelum memprosesnya
                if image is not None:
                    # Terapkan preprocessing
                    preprocessed_image = preprocess_image(image)
                    
                    # Simpan gambar yang telah diproses ke folder output
                    output_path = os.path.join(output_subfolder, filename)
                    cv2.imwrite(output_path, preprocessed_image)
            
                else:
                    print(f"Failed to load: {img_path}")

# Panggil fungsi untuk memproses gambar
process_and_save_images(input_folder, output_folder)


Balancing Data

UNDERSAMPLING

In [1]:
import os
import random
import shutil

# Path ke direktori kelas mayoritas dan minoritas
majority_class_dir = r'd:\SALSA NITIP\Nexmedis\preprocessing\PNEUMONIA\bacteria'
minority_class_dir = r'd:\SALSA NITIP\Nexmedis\preprocessing\PNEUMONIA\virus'
undersampled_dir = r'd:\SALSA NITIP\Nexmedis\preprocessing\PNEUMONIA\under_bacteria'

# Buat direktori untuk menyimpan hasil undersampling
os.makedirs(undersampled_dir, exist_ok=True)

# Hitung jumlah citra pada setiap kelas
num_minority = len(os.listdir(minority_class_dir))
num_majority = len(os.listdir(majority_class_dir))

# Tentukan jumlah citra yang ingin dipertahankan pada kelas mayoritas setelah undersampling
# Biasanya ini adalah jumlah yang mendekati (tetapi tidak lebih kecil dari) kelas minoritas
target_majority = 2000  # Contoh: 1,5x jumlah kelas minoritas

# Pilih citra secara acak dari kelas mayoritas
selected_files = random.sample(os.listdir(majority_class_dir), target_majority)

# Pindahkan citra yang dipilih ke direktori baru
for img_file in selected_files:
    shutil.copy(os.path.join(majority_class_dir, img_file), undersampled_dir)


UPSAMPLING

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Buat direktori untuk menyimpan hasil oversampling
minority_class_dir = r"D:\SALSA NITIP\Nexmedis\preprocessing\NORMAL"
oversampled_dir = r"D:\SALSA NITIP\Nexmedis\preprocessing\up_NORMAL"
os.makedirs(oversampled_dir, exist_ok=True)

existing_images = len(os.listdir(minority_class_dir))  # Jumlah gambar asli
augmented_images = len(os.listdir(oversampled_dir))  # Jumlah gambar hasil augmentasi yang sudah ada
target_total_images = 2000  # Target total 2000 gambar
required_additional_images = target_total_images - (existing_images + augmented_images)

# Inisialisasi ImageDataGenerator untuk augmentasi data
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

i = 0

# Hitung berapa banyak citra tambahan yang diperlukan
if required_additional_images > 0:
    # Dapatkan daftar file gambar asli
    original_images = os.listdir(minority_class_dir)
    num_original_images = len(original_images)
    
    # Hitung jumlah gambar yang perlu dihasilkan dari setiap gambar asli
    images_per_original = required_additional_images // num_original_images
    extra_images_needed = required_additional_images % num_original_images

    for img_file in original_images:
        img_path = os.path.join(minority_class_dir, img_file)
        img = plt.imread(img_path)  # Load image

        # Pastikan gambar memiliki 3 dimensi (jika grayscale, tambahkan dimensi channel)
        if len(img.shape) == 2:  # Jika gambar grayscale, tambahkan dimensi channel
            img = np.expand_dims(img, axis=-1)

        img = img.reshape((1,) + img.shape)  # Ubah dimensi citra untuk augmentasi, jadi (1, height, width, channels)

        # Hitung jumlah gambar yang harus dihasilkan dari gambar asli ini
        total_images_to_generate = images_per_original + (1 if extra_images_needed > 0 else 0)
        extra_images_needed -= 1

        # Hasilkan citra baru
        for batch in datagen.flow(img, batch_size=1, save_to_dir=oversampled_dir, save_prefix='image', save_format='jpeg'):
            i += 1  # Tambah counter setiap kali gambar baru dihasilkan
            final_image_count = existing_images + len(os.listdir(oversampled_dir))  # Total gambar saat ini

            if final_image_count >= target_total_images:
                break  # Berhenti jika jumlah gambar telah mencapai target
        if final_image_count >= target_total_images:
            break  # Berhenti loop luar jika jumlah gambar telah mencapai target

    # Pindahkan citra asli dari kelas minoritas ke direktori oversampled jika belum dipindahkan
    for img_file in original_images:
        dest_path = os.path.join(oversampled_dir, img_file)
        if not os.path.exists(dest_path):
            shutil.copy(os.path.join(minority_class_dir, img_file), dest_path)

print("Proses augmentasi selesai. Total gambar:", len(os.listdir(oversampled_dir)))


Proses augmentasi selesai. Total gambar: 2000


In [9]:
import os
import random
import shutil

# Tentukan path dari folder sumber dan folder tujuan
source_folder = r"D:\SALSA NITIP\Nexmedis\preprocessing\NORMAL"
destination_folder = r"D:\SALSA NITIP\Nexmedis\preprocessing\test\NORMAL"

# Buat daftar semua file dalam folder sumber
files = os.listdir(source_folder)

# Tentukan jumlah file yang ingin dipindahkan secara acak
num_files_to_move = 400  # Ganti dengan jumlah file yang ingin Anda pindahkan

# Pilih file secara acak dari daftar
files_to_move = random.sample(files, num_files_to_move)

# Pindahkan file yang dipilih secara acak ke folder tujuan
for file_name in files_to_move:
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, file_name)
    shutil.move(source_path, destination_path)

print(f"Moved {num_files_to_move} files from {source_folder} to {destination_folder} randomly.")


Moved 400 files from D:\SALSA NITIP\Nexmedis\preprocessing\NORMAL to D:\SALSA NITIP\Nexmedis\preprocessing\test\NORMAL randomly.


In [24]:
train_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\train"
test_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\test"

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.


In [29]:
model = tf.keras.Sequential([
    # First conv block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.BatchNormalization(),

    # Second conv block
    # tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.BatchNormalization(),

    # Third conv block
    # tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.BatchNormalization(),

    # Fourth conv block
    # tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.BatchNormalization(),

    # Fully connected layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

start_time = time.time()
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 111, 111, 64)     0         
 g2D)                                                            
                                                                 
 batch_normalization_16 (Bat  (None, 111, 111, 64)     256       
 chNormalization)                                                
                                                                 
 conv2d_33 (Conv2D)          (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 54, 54, 128)      0         
 g2D)                                                            
                                                      

Densenet 0.001 adam 32 batch size ada di file transferpneumo_nexsalsa

Densenet 0.0001 Adam 32 batch size

In [31]:
from keras.applications import DenseNet121
class CustomEarlyStopping(EarlyStopping):
    def on_train_end(self, logs=None):
        print("Training stopped early! Best epoch: ", self.best_epoch)
# Path ke direktori data
train_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\train"
test_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\test"

# Data augmentation dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
   
    class_mode='categorical',
    shuffle=False
)

# Load the Densenet model tanpa fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer VGG16
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan fully connected layers di atas VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

custom_early_stopping = CustomEarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
start_time = time.time()
# Melatih model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    callbacks=[custom_early_stopping]
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")


Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
        

Densenet 0.001 adam 64 batch size

In [32]:

class CustomEarlyStopping(EarlyStopping):
    def on_train_end(self, logs=None):
        print("Training stopped early! Best epoch: ", self.best_epoch)
# Path ke direktori data
train_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\train"
test_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\test"

# Data augmentation dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
   
    class_mode='categorical',
    shuffle=False
)

# Load the Densenet model tanpa fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer VGG16
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan fully connected layers di atas VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

custom_early_stopping = CustomEarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
start_time = time.time()
# Melatih model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    callbacks=[custom_early_stopping]
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")


Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_4[0][0]']       
        

Densenet 0.0001 adam 64 batch size

In [33]:

class CustomEarlyStopping(EarlyStopping):
    def on_train_end(self, logs=None):
        print("Training stopped early! Best epoch: ", self.best_epoch)
# Path ke direktori data
train_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\train"
test_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\test"

# Data augmentation dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
   
    class_mode='categorical',
    shuffle=False
)

# Load the Densenet model tanpa fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer VGG16
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan fully connected layers di atas VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

custom_early_stopping = CustomEarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
start_time = time.time()
# Melatih model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    callbacks=[custom_early_stopping]
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")


Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_6[0][0]']       
        

Densenet 0.001 adam 16 batch size

In [ ]:

class CustomEarlyStopping(EarlyStopping):
    def on_train_end(self, logs=None):
        print("Training stopped early! Best epoch: ", self.best_epoch)
# Path ke direktori data
train_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\train"
test_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\test"

# Data augmentation dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
   
    class_mode='categorical',
    shuffle=False
)

# Load the Densenet model tanpa fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer VGG16
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan fully connected layers di atas VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

custom_early_stopping = CustomEarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
start_time = time.time()
# Melatih model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    callbacks=[custom_early_stopping]
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")


Densenet 0.0001 adam 16 batch size

In [ ]:

class CustomEarlyStopping(EarlyStopping):
    def on_train_end(self, logs=None):
        print("Training stopped early! Best epoch: ", self.best_epoch)
# Path ke direktori data
train_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\train"
test_dir = r"d:\Documents\Nexmedis Salsa\chest_xray\preprocessing\test"

# Data augmentation dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
   
    class_mode='categorical',
    shuffle=False
)

# Load the Densenet model tanpa fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer VGG16
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan fully connected layers di atas VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

custom_early_stopping = CustomEarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
start_time = time.time()
# Melatih model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    callbacks=[custom_early_stopping]
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")


In [11]:
train_dir = r"D:\SALSA NITIP\Nexmedis\preprocessing\train"
test_dir = r"D:\SALSA NITIP\Nexmedis\preprocessing\test"

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.


In [14]:
from keras.applications import DenseNet121
class CustomEarlyStopping(EarlyStopping):
    def on_train_end(self, logs=None):
        print("Training stopped early! Best epoch: ", self.best_epoch)
# Path ke direktori data
train_dir = r"D:\SALSA NITIP\Nexmedis\preprocessing\train"
test_dir = r"D:\SALSA NITIP\Nexmedis\preprocessing\test"

# Data augmentation dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
   
    class_mode='categorical',
    shuffle=False
)

# Load the Densenet model tanpa fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer VGG16
for layer in base_model.layers:
    layer.trainable = False

# Menambahkan fully connected layers di atas VGG16
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Membuat model akhir
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

optimizerss = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizerss,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

custom_early_stopping = CustomEarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
start_time = time.time()
# Melatih model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,
    callbacks=[custom_early_stopping]
)
test_loss, test_accuracy = model.evaluate(test_generator)
end_time = time.time()

# Menghitung dan mencetak waktu komputasi
training_time = end_time - start_time
print(f"Waktu komputasi untuk pelatihan model: {training_time} detik")


Found 4800 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_3    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_3… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 13,468,611 (51.38 MB)

 Trainable params: 6,431,107 (24.53 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

Epoch 1/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 584s 3s/step - accuracy: 0.5508 - loss: 1.0689 - val_accuracy: 0.7608 - val_loss: 0.5614
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 541s 4s/step - accuracy: 0.6894 - loss: 0.6891 - val_accuracy: 0.7750 - val_loss: 0.5833
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 529s 3s/step - accuracy: 0.7200 - loss: 0.6381 - val_accuracy: 0.8050 - val_loss: 0.4845
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 572s 4s/step - accuracy: 0.7278 - loss: 0.6104 - val_accuracy: 0.8025 - val_loss: 0.4762
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 516s 3s/step - accuracy: 0.7431 - loss: 0.6074 - val_accuracy: 0.8092 - val_loss: 0.4739
Epoch 6/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 578s 4s/step - accuracy: 0.7554 - loss: 0.5855 - val_accuracy: 0.7975 - val_loss: 0.4820
Epoch 7/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 584s 4s/step - accuracy: 0.7665 - loss: 0.5586 - val_accuracy: 0.7767 - val_loss: 0.5262
Epoch 8/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 560s 4s/step - accuracy: 0.7701 - loss: 0.5633 - val_accu